In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import requests
from bs4 import BeautifulSoup

In [2]:
Title=[]
Brand=[]
Capacity=[]
Automation_Type=[]
Load_Type=[]
Current_Price=[]
Original_Price=[]
Ratings=[]
Rating_Reviews=[]
Discount=[]
for page in range(1,21):
    url = f"https://www.flipkart.com/search?q=washing+machine&sid=j9e%2Cabm%2C8qx&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&as-pos=2&as-type=RECENT&suggestionId=washing+machine%7CWashing+Machines&requestId=fca317df-781f-4bfe-9be6-911520a7751a&as-searchtext=washing&page={page}"
    req_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    req = requests.get(url,headers=req_header)
    soup = BeautifulSoup(req.text,"html.parser")
    #print(req)
    for i in soup.find_all("div",class_="tUxRFH"):
        product_link=i.find("a",class_="CGtC98").get("href")
        full_link="https://www.flipkart.com"+product_link
        p_req=requests.get(full_link,headers=req_header)
        soup_1=BeautifulSoup(p_req.text,"html.parser")

        # title
        title=soup_1.find("span",class_="VU-ZEz")
        if title:
            Title.append(title.text)
        else:
            Title.append(np.nan)
        #print(Title)

        # rating
        rating=soup_1.find("div",class_="XQDdHH")
        if rating:
            Ratings.append(rating.text)
        else:
            Ratings.append(np.nan)

        # Current_Price
        current_price=soup_1.find("div",class_="Nx9bqj CxhGGd")
        if current_price:
            Current_Price.append(current_price.text)
        else:
            Current_Price.append(np.nan)

        # original price
        original_price=soup_1.find("div",class_="yRaY8j A6+E6v")
        if original_price:
            Original_Price.append(original_price.text)
        else:
            Original_Price.append(np.nan)


        # rating and reviews
        rating_reviews=soup_1.find("span",class_="Wphh3N")
        if rating_reviews:
            Rating_Reviews.append(rating_reviews.text)
        else:
            Rating_Reviews.append(np.nan)

        # Discount
        discount=soup_1.find("div",class_="UkUFwK WW8yVX")
        if discount:
            Discount.append(discount.text)
        else:
            Discount.append(np.nan)

        # description
        #description=soup_1.find("span",class_="VU-ZEz").text
        # print(description)
        desc_tag = soup_1.find("span", class_="VU-ZEz")
        if desc_tag:
            description = desc_tag.text.strip()
        else:
            description = ""

        # brand
        brand_pattern=re.findall(r'^\w+',description)
        if brand_pattern:
            Brand.append(brand_pattern[0])
        else:
            Brand.append(np.nan)

        # capacity
        capacity_pattern=re.findall(r'\s(\d\.?\d*)\skg',description)
        if capacity_pattern:
            Capacity.append(capacity_pattern[0])
        else:
            Capacity.append(np.nan)
            
        # automatic type
        automation_pattern=re.findall(r'(?:Fully|Semi)\s+Automatic',description)
        if automation_pattern:
            Automation_Type.append(automation_pattern[0])
        else:
            Automation_Type.append(np.nan)


        # load type
        load_pattern=re.findall(r'(?:Top|Front)\s+Load',description)
        if load_pattern:
            Load_Type.append(load_pattern[0])
        else:
            Load_Type.append(np.nan)

        #print(Title)

doc={'Title':Title,'Brand':Brand,'Capacity':Capacity,'Automation_Type':Automation_Type,'Load_Type':Load_Type,'Current_Price':Current_Price,'Original_Price':Original_Price,'Ratings':Ratings,'Ratings and Reviews':Rating_Reviews,'Discount':Discount}
df=pd.DataFrame(doc)

In [74]:
del df["Title"]

In [75]:
df['number_rating'] = (df['Ratings and Reviews']
                .str.extract(r'([\d,]+)\s*Ratings')[0]
                .str.replace(',', '')
                .astype(float))

df['number_reviews'] = (df['Ratings and Reviews']
                 .str.extract(r'([\d,]+)\s*Reviews')[0]
                 .str.replace(',', '')
                 .astype(float))


In [76]:
del df["Ratings and Reviews"]

In [123]:
print(f'''the no of records in rating {len(Ratings)}
the no of records in current price {len(Current_Price)}\nthe no of records in original price {len(Original_Price)}
the no of records in Discount {len(Discount)}\nthe no of records in Brand {len(Brand)}
the no of records in Capacity {len(Capacity)}\nthe no of records in Automation_Type {len(Automation_Type)}\nthe no of records in Load_Type {len(Load_Type)}
the no of records in rating {len(df["number_rating"])}\nthe no of records in reviews {len(df["number_reviews"])}''')

the no of records in rating 480
the no of records in current price 480
the no of records in original price 480
the no of records in Discount 480
the no of records in Brand 480
the no of records in Capacity 480
the no of records in Automation_Type 480
the no of records in Load_Type 480
the no of records in rating 480
the no of records in reviews 480


In [78]:
df.head()

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews
0,Whirlpool,7,Semi Automatic,Top Load,"₹10,490","₹12,800",4.4,18% off,41641.0,2713.0
1,Godrej,7,Fully Automatic,Top Load,"₹13,990","₹27,300",4.2,48% off,9846.0,677.0
2,realme,7,Semi Automatic,Top Load,"₹7,990","₹12,490",4.2,36% off,94783.0,8182.0
3,realme,7.5,Semi Automatic,Top Load,"₹7,990","₹13,990",4.2,42% off,94783.0,8182.0
4,MarQ,7,Semi Automatic,Top Load,"₹7,490","₹13,990",4.3,46% off,62042.0,6102.0


In [79]:

# firstly we should clean up the commas and rupees symbols
df["Current_Price"] = (
    df["Current_Price"]
    .astype(str) 
    .str.replace("₹", "", regex=False)  
    .str.replace(",", "", regex=False)  
    .str.strip()               
)
df["Current_Price"] = pd.to_numeric(df["Current_Price"], errors="coerce").astype("Int32")
df["Original_Price"] = (
    df["Original_Price"]
    .astype(str)               
    .str.replace("₹", "", regex=False)  
    .str.replace(",", "", regex=False)  
    .str.strip()               
)
df["Original_Price"] = pd.to_numeric(df["Original_Price"], errors="coerce").astype("Int32")

df["Discount"] = (df["Discount"].astype(str).str.replace("% off","", regex = False).str.strip())
df["Discount"] = pd.to_numeric(df["Discount"], errors="coerce").astype("Int32")

In [80]:
df

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews
0,Whirlpool,7,Semi Automatic,Top Load,10490,12800,4.4,18,41641.0,2713.0
1,Godrej,7,Fully Automatic,Top Load,13990,27300,4.2,48,9846.0,677.0
2,realme,7,Semi Automatic,Top Load,7990,12490,4.2,36,94783.0,8182.0
3,realme,7.5,Semi Automatic,Top Load,7990,13990,4.2,42,94783.0,8182.0
4,MarQ,7,Semi Automatic,Top Load,7490,13990,4.3,46,62042.0,6102.0
...,...,...,...,...,...,...,...,...,...,...
475,LG,9,Fully Automatic,Top Load,24990,36990,4.4,32,85004.0,5943.0
476,Haier,6.5,Fully Automatic,Top Load,12990,18990,4.3,31,71997.0,8065.0
477,Whirlpool,7,Semi Automatic,Top Load,10490,12800,4.4,18,41641.0,2713.0
478,BOSCH,8.5,Semi Automatic,Top Load,14599,21000,4.1,30,377.0,29.0


In [2]:
df = pd.read_csv("anu.csv") 

In [3]:
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews
0,Whirlpool,7.0,Fully Automatic,Top Load,14490.0,19550.0,4.1,25.0,46397.0,2890.0
1,Godrej,7.0,Fully Automatic,Top Load,13990.0,27300.0,4.2,48.0,9824.0,678.0
2,realme,7.0,Semi Automatic,Top Load,7890.0,12490.0,4.2,36.0,94783.0,8182.0
3,realme,7.5,Semi Automatic,Top Load,8290.0,13990.0,4.2,40.0,94783.0,8182.0
4,MarQ,7.0,Semi Automatic,Top Load,7390.0,13990.0,4.3,47.0,62042.0,6102.0
...,...,...,...,...,...,...,...,...,...,...
475,LG,8.0,Semi Automatic,Top Load,13660.0,17990.0,4.4,24.0,92.0,4.0
476,Reliance,8.0,NaN,NaN,5990.0,8999.0,2.0,33.0,3.0,1.0
477,IFB,8.0,Fully Automatic,Front Load,32990.0,43790.0,4.3,24.0,28925.0,2208.0
478,Haier,8.5,Semi Automatic,Top Load,14808.0,20500.0,4.2,27.0,21.0,3.0


In [5]:
df.isna().sum()/len(df)*100

Brand              0.625000
Capacity           3.958333
Automation_Type    7.291667
Load_Type          7.291667
Current_Price      0.625000
Original_Price     1.250000
Ratings            0.625000
Discount           1.250000
number_rating      2.708333
number_reviews     2.708333
dtype: float64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            477 non-null    object 
 1   Capacity         461 non-null    float64
 2   Automation_Type  445 non-null    object 
 3   Load_Type        445 non-null    object 
 4   Current_Price    477 non-null    float64
 5   Original_Price   474 non-null    float64
 6   Ratings          477 non-null    float64
 7   Discount         474 non-null    float64
 8   number_rating    467 non-null    float64
 9   number_reviews   467 non-null    float64
dtypes: float64(7), object(3)
memory usage: 37.6+ KB


In [7]:
df

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews
0,Whirlpool,7.0,Fully Automatic,Top Load,14490.0,19550.0,4.1,25.0,46397.0,2890.0
1,Godrej,7.0,Fully Automatic,Top Load,13990.0,27300.0,4.2,48.0,9824.0,678.0
2,realme,7.0,Semi Automatic,Top Load,7890.0,12490.0,4.2,36.0,94783.0,8182.0
3,realme,7.5,Semi Automatic,Top Load,8290.0,13990.0,4.2,40.0,94783.0,8182.0
4,MarQ,7.0,Semi Automatic,Top Load,7390.0,13990.0,4.3,47.0,62042.0,6102.0
...,...,...,...,...,...,...,...,...,...,...
475,LG,8.0,Semi Automatic,Top Load,13660.0,17990.0,4.4,24.0,92.0,4.0
476,Reliance,8.0,NaN,NaN,5990.0,8999.0,2.0,33.0,3.0,1.0
477,IFB,8.0,Fully Automatic,Front Load,32990.0,43790.0,4.3,24.0,28925.0,2208.0
478,Haier,8.5,Semi Automatic,Top Load,14808.0,20500.0,4.2,27.0,21.0,3.0


In [8]:
import warnings
import numpy as np
warnings.filterwarnings("ignore")
df["Current_Price"] = df["Current_Price"].fillna(df["Current_Price"].median())
bins = [0, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, np.inf]
labels = ["0-10000","10000-15000","15000-20000","20000-25000",
          "25000-30000","30000-35000","35000-40000","40000-45000","45000+"]
df['Price_Range'] = pd.cut(df['Current_Price'], bins=bins, labels=labels)
columns = [c for c in df.select_dtypes(include=["float64"]).columns ]
for col in columns:
    df[col] = df.groupby("Price_Range")[col].transform(lambda x: x.fillna(x.median()))

In [9]:
df

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews,Price_Range
0,Whirlpool,7.0,Fully Automatic,Top Load,14490.0,19550.0,4.1,25.0,46397.0,2890.0,10000-15000
1,Godrej,7.0,Fully Automatic,Top Load,13990.0,27300.0,4.2,48.0,9824.0,678.0,10000-15000
2,realme,7.0,Semi Automatic,Top Load,7890.0,12490.0,4.2,36.0,94783.0,8182.0,0-10000
3,realme,7.5,Semi Automatic,Top Load,8290.0,13990.0,4.2,40.0,94783.0,8182.0,0-10000
4,MarQ,7.0,Semi Automatic,Top Load,7390.0,13990.0,4.3,47.0,62042.0,6102.0,0-10000
...,...,...,...,...,...,...,...,...,...,...,...
475,LG,8.0,Semi Automatic,Top Load,13660.0,17990.0,4.4,24.0,92.0,4.0,10000-15000
476,Reliance,8.0,NaN,NaN,5990.0,8999.0,2.0,33.0,3.0,1.0,0-10000
477,IFB,8.0,Fully Automatic,Front Load,32990.0,43790.0,4.3,24.0,28925.0,2208.0,30000-35000
478,Haier,8.5,Semi Automatic,Top Load,14808.0,20500.0,4.2,27.0,21.0,3.0,10000-15000


In [10]:
del df["Price_Range"]

In [11]:
df['Brand']=df['Brand'].fillna(df['Brand'].mode()[0])
df['Automation_Type']=df['Automation_Type'].fillna(df['Automation_Type'].mode()[0])
df['Load_Type']=df['Load_Type'].fillna(df['Load_Type'].mode()[0])


In [12]:
df

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews
0,Whirlpool,7.0,Fully Automatic,Top Load,14490.0,19550.0,4.1,25.0,46397.0,2890.0
1,Godrej,7.0,Fully Automatic,Top Load,13990.0,27300.0,4.2,48.0,9824.0,678.0
2,realme,7.0,Semi Automatic,Top Load,7890.0,12490.0,4.2,36.0,94783.0,8182.0
3,realme,7.5,Semi Automatic,Top Load,8290.0,13990.0,4.2,40.0,94783.0,8182.0
4,MarQ,7.0,Semi Automatic,Top Load,7390.0,13990.0,4.3,47.0,62042.0,6102.0
...,...,...,...,...,...,...,...,...,...,...
475,LG,8.0,Semi Automatic,Top Load,13660.0,17990.0,4.4,24.0,92.0,4.0
476,Reliance,8.0,Fully Automatic,Top Load,5990.0,8999.0,2.0,33.0,3.0,1.0
477,IFB,8.0,Fully Automatic,Front Load,32990.0,43790.0,4.3,24.0,28925.0,2208.0
478,Haier,8.5,Semi Automatic,Top Load,14808.0,20500.0,4.2,27.0,21.0,3.0


In [13]:
df.isnull().sum()

Brand              0
Capacity           0
Automation_Type    0
Load_Type          0
Current_Price      0
Original_Price     0
Ratings            0
Discount           0
number_rating      0
number_reviews     0
dtype: int64

In [14]:
numeric_col = df.select_dtypes(include = ["int", "float"]).columns
for i in numeric_col:
    q1 = df[i].quantile(.25)
    q3 = df[i].quantile(.75)
    IQR = q3 - q1
    upper_bound = q3 + 1.5*IQR
    lower_bound = q1 - 1.5*IQR
    outliers = df[i].where((df[i] < lower_bound) | (df[i] > upper_bound))
    outliers_count = outliers.count()
    print(f"The Percentage of Outliers in {i} is {round(outliers_count / len(outliers) * 100, 2)}%")

The Percentage of Outliers in Capacity is 1.25%
The Percentage of Outliers in Current_Price is 1.67%
The Percentage of Outliers in Original_Price is 0.42%
The Percentage of Outliers in Ratings is 6.67%
The Percentage of Outliers in Discount is 0.21%
The Percentage of Outliers in number_rating is 4.79%
The Percentage of Outliers in number_reviews is 6.46%


In [15]:
numeric_col = df.select_dtypes(include = ["int", "float"]).columns
for i in numeric_col:
    q1 = df[i].quantile(.25)
    q3 = df[i].quantile(.75)
    IQR = q3 - q1
    upper_bound = q3 + 1.5*IQR
    lower_bound = q1 - 1.5*IQR
    df[i] = np.where(df[i] > upper_bound, upper_bound, np.where(df[i] < lower_bound, lower_bound, df[i]))

In [16]:
df

,Brand,Capacity,Automation_Type,Load_Type,Current_Price,Original_Price,Ratings,Discount,number_rating,number_reviews
0,Whirlpool,7.0,Fully Automatic,Top Load,14490.0,19550.0,4.1,25.0,46397.0,2890.000
1,Godrej,7.0,Fully Automatic,Top Load,13990.0,27300.0,4.2,48.0,9824.0,678.000
2,realme,7.0,Semi Automatic,Top Load,7890.0,12490.0,4.2,36.0,71747.0,5475.375
3,realme,7.5,Semi Automatic,Top Load,8290.0,13990.0,4.2,40.0,71747.0,5475.375
4,MarQ,7.0,Semi Automatic,Top Load,7390.0,13990.0,4.3,47.0,62042.0,5475.375
...,...,...,...,...,...,...,...,...,...,...
475,LG,8.0,Semi Automatic,Top Load,13660.0,17990.0,4.4,24.0,92.0,4.000
476,Reliance,8.0,Fully Automatic,Top Load,5990.0,8999.0,3.8,33.0,3.0,1.000
477,IFB,8.0,Fully Automatic,Front Load,32990.0,43790.0,4.3,24.0,28925.0,2208.000
478,Haier,8.5,Semi Automatic,Top Load,14808.0,20500.0,4.2,27.0,21.0,3.000
